In [1]:
from pathlib import Path
from transformers import AutoTokenizer
import pandas as pd

In [21]:
tasks = ["NLU", "POL", "NLG"]
special_sep_tokens = [
    "[eoaa]",
    "[eoau]",
    "[eoda]",
    "[eodp]",
    "[eodu]",
    "[soaa]",
    "[soau]",
    "[soda]",
    "[sodp]",
    "[sodu]",
]
basedir = Path('/home/jitianbo/Workspace/driver_simulator_kvret/data/data_for_clm/')
csvdir = Path('/home/jitianbo/Workspace/driver_simulator_kvret/data/data_with_dp/')


modes = ["train", "test", "dev"]
csv_data = {
    mode: pd.read_csv(csvdir.joinpath(f"{mode}.csv"))
    for mode in modes
}

models = [
    (
        "bert-large-uncased", 
        "bert", 
        AutoTokenizer.from_pretrained("bert-large-uncased"),
    ),
    (
        "roberta-large", 
        "roberta",
        AutoTokenizer.from_pretrained("roberta-large"),
    ),
    
]

def remove_ss_token(txt):
    tokens = txt.split()
    tokens = [e for e in tokens if e not in special_sep_tokens]
    return " ".join(tokens)



SyntaxError: invalid syntax (882115108.py, line 59)

In [29]:
models[1][2].pad_token

'<pad>'

In [25]:
def remove_ss_token_in_history(txt,sep_token):
    # special_sep_tokens
    special_sep_tokens = [
        "[eoaa]",
        "[eoau]",
        "[eoda]",
        "[eodp]",
        "[eodu]",
        "[soaa]",
        "[soau]",
        "[soda]",
        "[sodp]",
        "[sodu]",
    ]
    tokens = txt.split()
    tokens = [e for e in tokens if e not in special_sep_tokens or not e.startswith("[so")]
    tokens = [sep_token if e in special_sep_tokens else e for e in tokens ]
    
    return " ".join(tokens)

In [ ]:
# # models = {
# #     "bert-large-uncased": "bert",
# #     "roberta-large": "roberta",
# # }


#     # tokenizer = AutoTokenizer.from_pretrained(modelname)
# for task in tasks:
#     dirpath = basedir.joinpath(task)
#     task_str = "DST" if task == "POL" else task
    
#     for mode in modes:
#         pd_data = csv_data[mode]
        
#         pd_data = pd_data[pd_data['task']]
#         source_fname = f"{mode}-{task}.source"
#         source_fpath = dirpath.joinpath(source_fname)
#         with source_fpath.open() as f:
#             sour_data = f.read().strip().splitlines()
#         sour_data = [remove_ss_token(e) for e in sour_data]
        
#         target_fname = f"{mode}-{task}.target"
#         target_fpath = dirpath.joinpath(target_fname)
#         with target_fpath.open() as f:
#             targ_data = f.read().strip().splitlines()
#         targ_data = [remove_ss_token(e) for e in targ_data]
    
#         for modelname, savename, tokenizer in models:
#             savedir = dirpath.joinpath(savename)
#             savedir.mkdir(exist_ok=True)
#             sep_token = tokenizer.sep_token
            
#             train_lines = []
#             sour_lines = []
#             targ_lines = []
#             for sour,targ in zip(sour_data,targ_data):
#                 if sour == "" or targ == "":
#                     continue
#                 train_txt = f"{sour} {sep_token} {targ} {sep_token}\n"
#                 train_lines.append(train_txt)
                
#                 sour_txt = f"{sour} {sep_token}\n"
#                 sour_lines.append(sour_txt)
                
#                 targ_txt = f"{targ}\n"
#                 targ_lines.append(targ_txt)
            
#             train_fname = f"{mode}.txt"
#             train_fpath = savedir.joinpath(train_fname)
#             with train_fpath.open('w') as f:
#                 f.writelines(train_lines)
            
#             sour_fname = f"{mode}.source"
#             sour_fpath = savedir.joinpath(sour_fname)
#             with sour_fpath.open('w') as f:
#                 f.writelines(sour_lines)
            
#             targ_fname = f"{mode}.target"
#             targ_fpath = savedir.joinpath(targ_fname)
#             with targ_fpath.open('w') as f:
#                 f.writelines(targ_lines)

In [14]:
for task in tasks:
    dirpath = basedir.joinpath(task)
    task_str = "DST" if task == "POL" else task
    
    for mode in modes:
        pd_data = csv_data[mode]
        
        cur_pd = pd_data[pd_data['task']==task_str]
        
        for modelname, savename, tokenizer in models:
            savedir = dirpath.joinpath(savename)
            savedir.mkdir(exist_ok=True)
            sep_token = tokenizer.sep_token
            
            train_lines = []
            sour_lines = []
            targ_lines = []
            for index,row in cur_pd.iterrows():
                history = row['history']
                if pd.isna(history):
                    history = ""
                input_ = row['input']
                profile = row['profile']
                target = row['target']
                    
                history = remove_ss_token(history)
                input_ = remove_ss_token(input_)
                profile = remove_ss_token(profile)
                target = remove_ss_token(target)
                if f"{history}{input_}" == "" or target=="":
                    continue
                
                if task == "POL":
                    sour_txt = f"{profile} {sep_token} {history} {sep_token} {input_} {sep_token}"
                elif history != "":
                    sour_txt = f"{history} {sep_token} {input_} {sep_token}"
                else:
                    sour_txt = f"{input_} {sep_token}"
                train_txt = f"{sour_txt} {target} {sep_token}"
                targ_txt = f"{target}"
                

                train_lines.append(f"{train_txt}\n")
                sour_lines.append(f"{sour_txt}\n")
                targ_lines.append(f"{targ_txt}\n")
            
#             train_fname = f"{mode}.txt"
#             train_fpath = savedir.joinpath(train_fname)
#             with train_fpath.open('w') as f:
#                 f.writelines(train_lines)
            
#             sour_fname = f"{mode}.source"
#             sour_fpath = savedir.joinpath(sour_fname)
#             with sour_fpath.open('w') as f:
#                 f.writelines(sour_lines)
            
#             targ_fname = f"{mode}.target"
#             targ_fpath = savedir.joinpath(targ_fname)
#             with targ_fpath.open('w') as f:
#                 f.writelines(targ_lines)